In [6]:
import pandas as pd

# Specify the path to your CSV file
csv_path = r"C:\Users\Administrator\ownCloud - xuwei@cloud.cts.shanghaitech.edu.cn\5. 应用\CNS\output_10.csv"

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_path, encoding='gbk')
df.head()

,Smiles,Target,Description
0,CC1OC(=O)CC(O)CC(O)CC(O)CCC(O)C(O)CC(=O)CC(O)C...,0,一种方法是通过观察分子结构，确定能够识别的结构片段。在这个分子中，可以找到以下结构片段：\n...
1,NCCCCC(NC(CCc1ccccc1)C(=O)O)C(=O)N2CCCC2C(=O)O,0,该化合物包含以下结构片段：\n\n1. NCCCCC- ：含有一个肽键（-CO-NH-）并连...
2,c1ccc(cc1)C2=NCC(=O)Nc3c2cc(cc3)[N+](=O)[O],1,'c1ccc(cc1)C2=NCC(=O)Nc3c2cc(cc3)[N+](=O)[O]'包...
3,CCN(CC)C(C)CN1c2ccccc2Sc3c1cccc3,1,'CCN(CC)C(C)CN1c2ccccc2Sc3c1cccc3'中包含以下结构片段：\n...
4,CC(CCCC(C)(C)O)C1CCC2C(=CC=C3CC(O)CC(O)C3=C)CC...,0,'CC(CCCC(C)(C)O)C1CCC2C(=CC=C3CC(O)CC(O)C3=C)C...


In [7]:
from datasets import Dataset

# Convert the pandas DataFrame to a datasets.Dataset
dataset = Dataset.from_pandas(df)

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["Description"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/699 [00:00<?, ? examples/s]

In [28]:
tokenized_datasets

Dataset({
    features: ['Smiles', 'Target', 'Description', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 699
})

In [12]:
from sklearn.model_selection import train_test_split

# Assuming your tokenized dataset has a 'train' split
train_data, dev_data = train_test_split(tokenized_datasets, test_size=0.2, random_state=42)

In [32]:
train_data.keys()

dict_keys(['Smiles', 'Target', 'Description', 'input_ids', 'token_type_ids', 'attention_mask'])

In [35]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Assuming train_data and test_data are dictionaries containing your data
train_dataset = TensorDataset(
    torch.tensor(train_data['input_ids']),
    torch.tensor(train_data['token_type_ids']),
    torch.tensor(train_data['attention_mask']),
    torch.tensor(train_data['Target'])
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Similarly, create a DataLoader for the validation set
dev_dataset = TensorDataset(
    torch.tensor(dev_data['input_ids']),
    torch.tensor(dev_data['token_type_ids']),
    torch.tensor(dev_data['attention_mask']),
    torch.tensor(dev_data['Target'])
)

dev_loader = DataLoader(dev_dataset, batch_size=32, shuffle=True)

In [37]:
# define model
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
# define optim and loss function
import torch.optim as optim
import torch.nn as nn

optimizer = optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

In [ ]:
# training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        input_ids, token_type_ids, attention_mask, target = batch
        input_ids, token_type_ids, attention_mask, target = (
            input_ids.to(device),
            token_type_ids.to(device),
            attention_mask.to(device),
            target.to(device),
        )

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        loss = criterion(outputs.logits, target)
        loss.backward()
        optimizer.step()

# Save or serialize your trained model if needed
# ...

In [1]:
# evaluation
from sklearn.metrics import fbeta_score
model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for batch in validation_loader:  # Use your validation DataLoader here
        input_ids, token_type_ids, attention_mask, target = batch
        input_ids, token_type_ids, attention_mask, target = (
            input_ids.to(device),
            token_type_ids.to(device),
            attention_mask.to(device),
            target.to(device),
        )

        outputs = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()

        all_preds.extend(preds)
        all_targets.extend(target.cpu().numpy())

# Compute F2-score
f2 = fbeta_score(all_targets, all_preds, beta=2)
print(f'F2-Score: {f2}')

NameError: name 'model' is not defined